In [1]:
# 导入进行线性回归的包
import pandas as pd

test_df = pd.read_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/衡阳站多模型验证期结果.csv",parse_dates=["Date"])
test_df_copy = test_df
test_df_copy = test_df_copy.iloc[:, :2]
test_df.reset_index(drop=True)
test_df = test_df.set_index('Date')
test_df_copy

,Date,Qobs
0,1996-11-25,391
1,1996-11-26,383
2,1996-11-27,292
3,1996-11-28,316
4,1996-11-29,309
...,...,...
1858,2001-12-27,766
1859,2001-12-28,653
1860,2001-12-29,536
1861,2001-12-30,483


In [2]:
"""
读取训练数据
"""
train_df = pd.read_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/衡阳站多模型率定期结果.csv",parse_dates=["Date"])
train_df_copy = train_df
train_df_copy = train_df_copy.iloc[:, :2]
train_df.reset_index(drop=True)
train_df = train_df.set_index('Date')
train_df.head()

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,RNN,LSTM
Date,,,,,,,,,
1985-01-16,479,125.424924,89.490725,203.925076,1228.700,535.80830,566.903486,465.21158,473.53240
1985-01-17,542,129.779407,108.622667,176.088411,1206.218,494.26337,503.363165,410.13797,441.47990
1985-01-18,557,151.626513,151.282451,159.980960,1213.434,559.14910,633.332359,493.82303,537.95306
1985-01-19,476,201.089959,153.655796,147.003401,1115.715,556.46497,597.860601,518.53470,512.38910
1985-01-20,464,175.316247,130.615103,129.756714,1002.941,462.31903,477.763243,443.38382,425.26486


In [3]:
# 缺失值检验
test_df[test_df.isnull() == True].count()
train_df[test_df.isnull() == True].count()

Qobs    0
XAJ     0
TVGM    0
TANK    0
GR4J    0
BP      0
SVR     0
RNN     0
LSTM    0
dtype: int64

In [4]:
# 计算各特征值相关系数
test_df.corr()

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,RNN,LSTM
Qobs,1.000000,0.896217,0.842456,0.858592,0.888339,0.968933,0.942743,0.970399,0.972495
XAJ,0.896217,1.000000,0.914469,0.919943,0.930146,0.909211,0.864139,0.915859,0.909286
TVGM,0.842456,0.914469,1.000000,0.963266,0.867713,0.859441,0.800942,0.870563,0.864809
TANK,0.858592,0.919943,0.963266,1.000000,0.880701,0.876754,0.784067,0.882521,0.883572
GR4J,0.888339,0.930146,0.867713,0.880701,1.000000,0.888147,0.824780,0.893655,0.907198
BP,0.968933,0.909211,0.859441,0.876754,0.888147,1.000000,0.954180,0.983043,0.988287
SVR,0.942743,0.864139,0.800942,0.784067,0.824780,0.954180,1.000000,0.958204,0.956806
RNN,0.970399,0.915859,0.870563,0.882521,0.893655,0.983043,0.958204,1.000000,0.989088
LSTM,0.972495,0.909286,0.864809,0.883572,0.907198,0.988287,0.956806,0.989088,1.000000


In [5]:
train_df.corr()

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,RNN,LSTM
Qobs,1.000000,0.911282,0.843187,0.858726,0.893290,0.971987,0.953831,0.973566,0.977645
XAJ,0.911282,1.000000,0.885245,0.893518,0.907303,0.921369,0.894431,0.923146,0.922956
TVGM,0.843187,0.885245,1.000000,0.958303,0.853625,0.864263,0.811132,0.864368,0.865490
TANK,0.858726,0.893518,0.958303,1.000000,0.848591,0.872021,0.798730,0.873861,0.877417
GR4J,0.893290,0.907303,0.853625,0.848591,1.000000,0.888781,0.836994,0.883121,0.902848
BP,0.971987,0.921369,0.864263,0.872021,0.888781,1.000000,0.967798,0.981936,0.991908
SVR,0.953831,0.894431,0.811132,0.798730,0.836994,0.967798,1.000000,0.973795,0.971496
RNN,0.973566,0.923146,0.864368,0.873861,0.883121,0.981936,0.973795,1.000000,0.988673
LSTM,0.977645,0.922956,0.865490,0.877417,0.902848,0.991908,0.971496,0.988673,1.000000


In [6]:
# # 通过加入一个参数kind='reg'，seaborn可以添加一条最佳拟合直线和95%的置信带。
# from matplotlib import pyplot as plt
# import seaborn as sns
# 
# sns.pairplot(df, x_vars=['LSTM', 'BP', 'RNN', 'SVR'], y_vars='Qobs', size=7, aspect=0.8, kind='reg')
# plt.savefig("pairplot.jpg")
# plt.show()

In [7]:
# from matplotlib import pyplot as plt
# import seaborn as sns
# sns.pairplot(df, x_vars=['XAJ', 'TVGM'], y_vars='Qobs', size=7, aspect=0.8, kind='reg')
# plt.savefig("pairplot02.jpg")
# plt.show()

In [8]:
from matplotlib import pyplot as plt
# import seaborn as sns
# 
# sns.pairplot(df, x_vars=['XAJ', 'TVGM', 'RNN', 'LSTM'], y_vars='Qobs', size=7, aspect=0.8, kind='reg')
# plt.savefig("pairplot03.jpg")
# plt.show()

In [9]:
# 分割数据集和测试集
# test_split = round(len(df) * 0.20)
# df_for_training = df[:-test_split]
# df_for_testing = df[-test_split:]
# df_for_training.shape

In [10]:
# 划分数据
# df_for_training = pd.DataFrame(df_for_training)
# trainX = df_for_training.iloc[:, 1:]
# trainY = df_for_training.iloc[:, :1]
# testX = df_for_testing.iloc[:, 1:]
# testY = df_for_testing.iloc[:, :1]

In [11]:
"""
方案一：两个概念性组合(XAJ+GR4J)
"""
from sklearn.linear_model import LinearRegression
from tool import Indicator

# 处理数据
trainX_XAJ = train_df.iloc[:, 1]
trainX_GR4J = train_df.iloc[:, 4]
trainY1 = train_df.iloc[:, :1]
trainX1 = pd.merge(trainX_XAJ, trainX_GR4J, on="Date", how="left")
testX_XAJ = test_df.iloc[:, 1]
testX_GR4J = test_df.iloc[:, 4]
testY1 = test_df.iloc[:, :1]
testX1 = pd.merge(testX_XAJ, testX_GR4J, on="Date", how="left")

#线性拟合
model = LinearRegression()
model.fit(trainX1, trainY1)
# 截距
a = model.intercept_
# 回归系数
b = model.coef_
print("最佳拟合线：截距", str(a), "回归系数：", str(b))

#结果输出
train_pred1 = model.predict(trainX1)
train_pred1 = pd.DataFrame(train_pred1)
train_pred1.columns = ["MLR1"]
test_pred1 = model.predict(testX1)
test_pred1 = pd.DataFrame(test_pred1)
test_pred1.columns = ["MLR1"]
# 数据输出

#预报评估
Indicator.eval(train_pred1["MLR1"], train_df_copy["Qobs"])
Indicator.eval(test_pred1["MLR1"], test_df_copy["Qobs"])

最佳拟合线：截距 [116.56415633] 回归系数： [[0.60404233 0.4027121 ]]
MAE:300.5480642654302
RMSE:512.5583845281658
NSE:0.8554335577759659
RE:-1.2320177309953779e-14
Qmaxe:0.17488853154308073
Qmine:0.4990524230616422
MAE:394.5108661322264
RMSE:631.6009425735352
NSE:0.8166694395824623
RE:8.18134606485603
Qmaxe:0.027589964101864393
Qmine:0.7458260835071547


(394.5108661322264,
 631.6009425735352,
 0.8166694395824623,
 8.18134606485603,
 0.027589964101864393,
 0.7458260835071547)

In [12]:
"""
方案二：两个黑箱组合(RNN+LSTM)
"""
from sklearn.linear_model import LinearRegression
from tool import Indicator

# 处理数据
trainX_LSTM = train_df.iloc[:, 8]
trainX_RNN = train_df.iloc[:, 7]
trainY2 = train_df.iloc[:, :1]
trainX2 = pd.merge(trainX_RNN, trainX_LSTM, on="Date", how="left")
testX_RNN = test_df.iloc[:, 7]
testX_LSTM = test_df.iloc[:, 8]
testY2 = test_df.iloc[:, :1]
testX2 = pd.merge(testX_RNN, testX_LSTM, on="Date", how="left")

#线性拟合
model = LinearRegression()
model.fit(trainX2, trainY2)
# 截距
a = model.intercept_
# 回归系数
b = model.coef_
print("最佳拟合线：截距", str(a), "回归系数：", str(b))

#结果输出
train_pred2 = model.predict(trainX2)
train_pred2 = pd.DataFrame(train_pred2)
train_pred2.columns = ["MLR2"]
test_pred2 = model.predict(testX2)
test_pred2 = pd.DataFrame(test_pred2)
test_pred2.columns = ["MLR2"]
# 数据输出

#预报评估
Indicator.eval(train_pred2["MLR2"], train_df_copy["Qobs"])
Indicator.eval(test_pred2["MLR2"], test_df_copy["Qobs"])

最佳拟合线：截距 [3.73731687] 回归系数： [[0.30489382 0.69338057]]
MAE:136.5108646795132
RMSE:276.39475384491533
NSE:0.9579621593140236
RE:-1.8334168995530632e-14
Qmaxe:0.04291521972692458
Qmine:0.11218638442473196
MAE:173.97811084976357
RMSE:333.5750632145279
NSE:0.9488628545869565
RE:-0.3455381007692564
Qmaxe:0.20793337244932664
Qmine:0.1580586996600785


(173.97811084976357,
 333.5750632145279,
 0.9488628545869565,
 -0.3455381007692564,
 0.20793337244932664,
 0.1580586996600785)

In [13]:
"""
方案三：LSTM+BP+GR4J+XAJ
"""
from sklearn.linear_model import LinearRegression
from tool import Indicator

# 处理数据
trainY3 = train_df.iloc[:, :1]
trainX3 = pd.merge(trainX1, trainX2, on="Date", how="left")
testY3 = test_df.iloc[:, :1]
testX3 = pd.merge(testX1, testX2, on="Date", how="left")

#线性拟合
model = LinearRegression()
model.fit(trainX3, trainY3)
# 截距
a = model.intercept_
# 回归系数
b = model.coef_
print("最佳拟合线：截距", str(a), "回归系数：", str(b))

#结果输出
train_pred3 = model.predict(trainX3)
train_pred3 = pd.DataFrame(train_pred3)
train_pred3.columns = ["MLR3"]
test_pred3 = model.predict(testX3)
test_pred3 = pd.DataFrame(test_pred3)
test_pred3.columns = ["MLR3"]
# 数据输出

#预报评估
print("---------------训练期-----------------")
Indicator.eval(train_pred3["MLR3"], train_df_copy["Qobs"])
print("---------------验证期-----------------")
Indicator.eval(test_pred3["MLR3"], test_df_copy["Qobs"])

最佳拟合线：截距 [-2.24942479] 回归系数： [[-0.00232608  0.08140669  0.33739658  0.59069113]]
---------------训练期-----------------
MAE:137.06943406934784
RMSE:273.02281545712145
NSE:0.9589816023948833
RE:-3.760495965789985e-15
Qmaxe:0.03520713273461063
Qmine:0.05092981953751632
---------------验证期-----------------
MAE:174.33386840093405
RMSE:332.88381956632475
NSE:0.9490745706655961
RE:0.09894780119686666
Qmaxe:0.21836180914245762
Qmine:0.050704525858907126


(174.33386840093405,
 332.88381956632475,
 0.9490745706655961,
 0.09894780119686666,
 0.21836180914245762,
 0.050704525858907126)

In [14]:
test_df_full = pd.concat([test_df_copy, test_pred1, test_pred2, test_pred3], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/HY_MLRTest.csv")
train_df_full = pd.concat([train_df_copy, train_pred1, train_pred2, train_pred3], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/HY_MLRTrain.csv")
test_df_full

,Qobs,MLR1,MLR2,MLR3
Date,,,,
1996-11-25,391,548.628556,353.888570,381.063034
1996-11-26,383,542.871361,355.349390,377.858110
1996-11-27,292,462.304761,350.998377,359.734683
1996-11-28,316,404.792903,266.541230,273.205044
1996-11-29,309,362.744247,324.180252,319.284280
...,...,...,...,...
2001-12-27,766,430.618203,645.448699,618.165371
2001-12-28,653,413.964084,665.841552,637.363915
2001-12-29,536,398.944663,556.571217,537.713174
